In [5]:
import tkinter as tk
from tkinter import messagebox
from pymongo import MongoClient
from bson.objectid import ObjectId

# Connect to MongoDB (update with your MongoDB URI)
client = MongoClient('mongodb://localhost:27017/')
db = client['swimming_db']
members_collection = db['members']
classes_collection = db['classes']

# Function for the Main Window
def main_window():
    root = tk.Tk()
    root.title("Swimming Management System")

    add_member_button = tk.Button(root, text="Add Member", command=add_member_window)
    add_member_button.grid(row=0, column=0, padx=20, pady=20)

    add_class_button = tk.Button(root, text="Add Swimming Class", command=add_class_window)
    add_class_button.grid(row=0, column=1, padx=20, pady=20)

    allot_class_button = tk.Button(root, text="Allot Class", command=allot_class_window)
    allot_class_button.grid(row=0, column=2, padx=20, pady=20)
    
    view_classes_button = tk.Button(root, text="View Member Classes", command=view_member_classes_window)
    view_classes_button.grid(row=0, column=3, padx=20, pady=20)
    
    view_all_members_button = tk.Button(root, text="View All Members", command=view_all_members_window)
    view_all_members_button.grid(row=1, column=0, padx=20, pady=20)
    
    view_all_classes_button = tk.Button(root, text="View All Classes", command=view_all_classes_window)
    view_all_classes_button.grid(row=1, column=1, padx=20, pady=20)

    root.mainloop()

# Function to display the Add Member Window
def add_member_window():
    add_member_win = tk.Toplevel()
    add_member_win.title("Add Member")

    tk.Label(add_member_win, text="Member Name").grid(row=0, column=0, padx=10, pady=5)
    name_entry = tk.Entry(add_member_win)
    name_entry.grid(row=0, column=1, padx=10, pady=5)

    tk.Label(add_member_win, text="Member ID").grid(row=1, column=0, padx=10, pady=5)
    member_id_entry = tk.Entry(add_member_win)
    member_id_entry.grid(row=1, column=1, padx=10, pady=5)

    def add_member():
        name = name_entry.get()
        member_id = member_id_entry.get()

        if not name or not member_id:
            messagebox.showerror("Input Error", "All fields are required.")
            return

        member = {"name": name, "member_id": member_id}
        members_collection.insert_one(member)
        messagebox.showinfo("Success", "Member added successfully.")
        add_member_win.destroy()

    add_member_button = tk.Button(add_member_win, text="Add Member", command=add_member)
    add_member_button.grid(row=2, column=0, columnspan=2, pady=10)

# Function to display the Add Class Window
def add_class_window():
    add_class_win = tk.Toplevel()
    add_class_win.title("Add Swimming Class")

    tk.Label(add_class_win, text="Class Name").grid(row=0, column=0, padx=10, pady=5)
    class_name_entry = tk.Entry(add_class_win)
    class_name_entry.grid(row=0, column=1, padx=10, pady=5)

    tk.Label(add_class_win, text="Class ID").grid(row=1, column=0, padx=10, pady=5)
    class_id_entry = tk.Entry(add_class_win)
    class_id_entry.grid(row=1, column=1, padx=10, pady=5)

    def add_class():
        class_name = class_name_entry.get()
        class_id = class_id_entry.get()

        if not class_name or not class_id:
            messagebox.showerror("Input Error", "All fields are required.")
            return

        swimming_class = {"class_name": class_name, "class_id": class_id, "members": []}
        classes_collection.insert_one(swimming_class)
        messagebox.showinfo("Success", "Swimming class added successfully.")
        add_class_win.destroy()

    add_class_button = tk.Button(add_class_win, text="Add Class", command=add_class)
    add_class_button.grid(row=2, column=0, columnspan=2, pady=10)

# Function to display the Allot Class Window
def allot_class_window():
    allot_class_win = tk.Toplevel()
    allot_class_win.title("Allot Class")

    tk.Label(allot_class_win, text="Select Member").grid(row=0, column=0, padx=10, pady=5)
    member_listbox = tk.Listbox(allot_class_win, width=40, height=10)
    member_listbox.grid(row=1, column=0, columnspan=2, padx=10, pady=5)

    members = members_collection.find()
    for member in members:
        member_listbox.insert(
            tk.END,
            f"{member['name']} - {member['member_id']} - {member['_id']}"
        )

    tk.Label(allot_class_win, text="Class ID").grid(row=2, column=0, padx=10, pady=5)
    class_id_entry = tk.Entry(allot_class_win)
    class_id_entry.grid(row=2, column=1, padx=10, pady=5)

    def allot_class():
        try:
            selected_item = member_listbox.get(member_listbox.curselection())
            member_id = selected_item.split(" - ")[-1]

            class_id = class_id_entry.get()
            swimming_class = classes_collection.find_one({"class_id": class_id})

            if not swimming_class:
                messagebox.showerror("Class Not Found", "Class ID not found.")
                return

            classes_collection.update_one(
                {"_id": swimming_class["_id"]},
                {"$addToSet": {"members": ObjectId(member_id)}}
            )
            messagebox.showinfo("Success", f"Member allotted to class {swimming_class['class_name']}.")
            allot_class_win.destroy()
        except Exception as e:
            messagebox.showerror("Selection Error", "Please select a member and enter a valid class ID.")

    allot_button = tk.Button(allot_class_win, text="Allot Class", command=allot_class)
    allot_button.grid(row=3, column=0, columnspan=2, pady=10)

# Function to view all members
def view_all_members_window():
    view_win = tk.Toplevel()
    view_win.title("All Members")

    result_text = tk.Text(view_win, height=20, width=50)
    result_text.grid(row=0, column=0, padx=10, pady=10)

    members = members_collection.find()
    result_text.insert(tk.END, "List of Members:\n")
    result_text.insert(tk.END, "-------------------------------\n")
    for member in members:
        result_text.insert(tk.END, f"Name: {member['name']}, ID: {member['member_id']}\n")

# Function to view all classes
def view_all_classes_window():
    view_win = tk.Toplevel()
    view_win.title("All Classes")

    result_text = tk.Text(view_win, height=20, width=50)
    result_text.grid(row=0, column=0, padx=10, pady=10)

    classes = classes_collection.find()
    result_text.insert(tk.END, "List of Classes:\n")
    result_text.insert(tk.END, "-------------------------------\n")
    for swimming_class in classes:
        result_text.insert(tk.END, f"Class Name: {swimming_class['class_name']}, ID: {swimming_class['class_id']}\n")
        result_text.insert(tk.END, "Members:\n")
        for member_id in swimming_class["members"]:
            member = members_collection.find_one({"_id": member_id})
            if member:
                result_text.insert(tk.END, f"- {member['name']} (ID: {member['member_id']})\n")
        result_text.insert(tk.END, "-------------------------------\n")

# Function to view a member's classes
def view_member_classes_window():
    view_win = tk.Toplevel()
    view_win.title("View Member Classes")

    tk.Label(view_win, text="Select Member").grid(row=0, column=0, padx=10, pady=5)
    member_listbox = tk.Listbox(view_win, width=40, height=10)
    member_listbox.grid(row=1, column=0, columnspan=2, padx=10, pady=5)

    members = members_collection.find()
    for member in members:
        member_listbox.insert(
            tk.END,
            f"{member['name']} - {member['member_id']} - {member['_id']}"
        )

    result_text = tk.Text(view_win, height=10, width=50)
    result_text.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

    def view_classes():
        try:
            selected_item = member_listbox.get(member_listbox.curselection())
            member_id = selected_item.split(" - ")[-1]

            # Find classes that include the selected member
            swimming_classes = classes_collection.find({"members": ObjectId(member_id)})

            result_text.delete(1.0, tk.END)  # Clear previous results
            result_text.insert(tk.END, f"Classes for {selected_item}:\n")
            result_text.insert(tk.END, "-------------------------------\n")
            for swimming_class in swimming_classes:
                result_text.insert(tk.END, f"Class Name: {swimming_class['class_name']}, ID: {swimming_class['class_id']}\n")
            if swimming_classes.count() == 0:
                result_text.insert(tk.END, "No classes found for this member.\n")
        except Exception as e:
            messagebox.showerror("Selection Error", "Please select a member.")

    view_classes_button = tk.Button(view_win, text="View Classes", command=view_classes)
    view_classes_button.grid(row=3, column=0, columnspan=2, pady=10)

# Start the main window
if __name__ == "__main__":
    main_window()
